# Graph from the New Pope Project

Inspiration and data from: https://www.the-new-pope.org/index.html

As the authors of "The New Pope Project" noted the election of Cardinal Prevost was a surprise as he was not part of the main component. We added the missing link, Cardinal Green, to join the components.

We then visualized the network using the [yFiles Graphs for Jupyter](https://www.yworks.com/products/yfiles-graphs-for-jupyter) plugin.
Where available the papbility-index, an informal measure of how good a fit as a pope a given person would be, is used as a heatmap.

Gist of the Data (from "The New Pope Project"):
https://gist.githubusercontent.com/richirikken/3d11c9dcef8eb7bda898437a4ed395a2/raw/9eeb8666ea83a3039e6ae3af38cac8acdd9b63a5/network-dd3927e2-243.gexf


In [2]:
# @title
%pip install yfiles_jupyter_graphs --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00


In [17]:
# @title
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
# @title
from google.colab import output
output.disable_custom_widget_manager()

Parse the data

In [39]:
# @title  {"display-mode":"form"}
import xml.etree.ElementTree as ET
import requests
from io import StringIO
from yfiles_jupyter_graphs import GraphWidget


# GEXF files often use namespaces. These are the common ones.
# The 'default' GEXF namespace (often not prefixed in the XML but needs to be handled by parsers)
GEXF_NS = 'http://gexf.net/1.3'
# The 'viz' namespace for visualization attributes
VIZ_NS = 'http://gexf.net/1.3/viz'

# Helper to construct fully qualified names for ElementTree find/findall
# e.g., for a tag 'node' in the GEXF_NS, it becomes '{http://www.gexf.net/1.2draft}node'
def _q(ns_uri, tag_name):
    return f"{{{ns_uri}}}{tag_name}"

def try_convert_value(value_str):
    """
    Tries to convert a string value to int, then float.
    If both fail, returns the original string.
    """
    if value_str is None:
        return None
    try:
        return int(value_str)
    except ValueError:
        try:
            return float(value_str)
        except ValueError:
            return value_str

def parse_gexf(path):
    """
    Parses a GEXF file into lists of nodes and edges with their properties.

    Args:
        gexf_file_path (str): Path to the GEXF file.

    Returns:
        tuple: (nodes_list, edges_list)
            nodes_list: [{id:str, properties:map<str, any>}]
            edges_list: [{id:str, source:str, target:str, properties:map<str, any>}]
                        (Note: GEXF edges can also have properties, so including it)
    """
    nodes_list = []
    edges_list = []

    try:
        response = requests.get(path)
        response.raise_for_status()

        content_io = StringIO(response.content.decode('utf-8'))

        # Parse GEXF 1.3 file
        tree = ET.parse(content_io)
        root = tree.getroot()
    except ET.ParseError as e:
        print(f"Error parsing XML in {gexf_file_path}: {e}")
        return [], []
    except FileNotFoundError:
        print(f"Error: File not found at {gexf_file_path}")
        return [], []

    # GEXF structure is typically <gexf><graph><nodes>...</nodes><edges>...</edges></graph></gexf>
    graph_element = root.find(_q(GEXF_NS, 'graph'))
    if graph_element is None:
        # Try finding graph without namespace if root itself is <graph> (less common for full GEXF)
        if root.tag == _q(GEXF_NS, 'graph'):
            graph_element = root
        else:
            print("Error: <graph> element not found. The GEXF file might be malformed or use unexpected namespaces.")
            return [], []

    # --- Parse Nodes ---
    nodes_container = graph_element.find(_q(GEXF_NS, 'nodes'))
    if nodes_container is not None:
        for node_element in nodes_container.findall(_q(GEXF_NS, 'node')):
            node_id = node_element.get('id')
            properties = {}

            # Get 'label' attribute if present
            label = node_element.get('label')
            if label is not None:
                properties['label'] = label

            # Parse <attvalues>
            attvalues_element = node_element.find(_q(GEXF_NS, 'attvalues'))
            if attvalues_element is not None:
                for attvalue_element in attvalues_element.findall(_q(GEXF_NS, 'attvalue')):
                    attr_for = attvalue_element.get('for')
                    attr_value_str = attvalue_element.get('value')
                    if attr_for: # Ensure 'for' attribute exists
                        properties[attr_for] = try_convert_value(attr_value_str)



            nodes_list.append({'id': node_id, 'properties': properties})

    # --- Add Prevost nodes ---
    nodes_list.append({'id': 'n909', 'properties':{
        "label": "Robert Francis Cardinal Prevost",
        "v_name": "bprevost.html",
        "v_wikidata_id": "Q6109517",
        "v_degree": 0.00391756089252257,
        "v_betweenness" :6.30253780421887E-4,
        "v_closeness" :0.194042862332001,
        "v_pagerank" :3.27612439416517E-4,
        "modularity_class":22,
        "v_country_of_citizenship":"US/PE"
    }})

    nodes_list.append({'id':"n1473", 'properties':{
        "label": "Archbishop James Patrick Green",
        "v_name": "bgreen.html",
        "v_wikidata_id" :"Q553470",
        "v_degree" :0.00749446431613013,
        "v_betweenness" :0.00597659631143183,
        "v_closeness" :0.211732065001982,
        "v_pagerank" :3.8021888852157E-4,
        "modularity_class" :22,
        "v_country_of_citizenship":"US"

    }})


    # --- Parse Edges ---
    edges_container = graph_element.find(_q(GEXF_NS, 'edges'))
    if edges_container is not None:
        for edge_element in edges_container.findall(_q(GEXF_NS, 'edge')):
            edge_id = edge_element.get('id')
            source_id = edge_element.get('source')
            target_id = edge_element.get('target')

            edge_properties = {} # Edges can also have properties

            # Get 'label' attribute if present
            label = edge_element.get('label')
            if label is not None:
                edge_properties['label'] = label

            # Get 'type' attribute if present (e.g. directed, undirected)
            edge_type = edge_element.get('type')
            if edge_type is not None:
                edge_properties['type'] = edge_type

            # Get 'weight' attribute if present
            weight = edge_element.get('weight')
            if weight is not None:
                edge_properties['weight'] = try_convert_value(weight)

            # Parse <attvalues> for edges
            attvalues_element = edge_element.find(_q(GEXF_NS, 'attvalues'))
            if attvalues_element is not None:
                for attvalue_element in attvalues_element.findall(_q(GEXF_NS, 'attvalue')):
                    attr_for = attvalue_element.get('for')
                    attr_value_str = attvalue_element.get('value')
                    if attr_for:
                        edge_properties[attr_for] = try_convert_value(attr_value_str)

            edges_list.append({
                'id': edge_id,
                'start': source_id,
                'end': target_id,
                'properties': edge_properties
            })

    #--- Add Prevost Edges ---
    edges_list.append({
                'id': "23876",
                'start': "n1473",
                'end': "n909",
                'properties': {"e_relation": "consecrator"}
            })
    edges_list.append({
                'id': "21958",
                'start': "n61",
                'end': "n1473",
                'properties': {"e_relation": "co_consecrator"}
            })
    return nodes_list, edges_list

In [38]:
# @title
nodes, edges = parse_gexf("https://gist.githubusercontent.com/richirikken/3d11c9dcef8eb7bda898437a4ed395a2/raw/9eeb8666ea83a3039e6ae3af38cac8acdd9b63a5/network-dd3927e2-243.gexf")

filtered_edges = [edge for edge in edges if edge["properties"]["e_relation"] != "common_consecration" or edge["start"] < edge["end"]]


w = GraphWidget()
w.nodes = nodes
w.edges = filtered_edges
w.directed = True

def name_country_label_mapping(index, node):
    return "{name}\n{country}".format(name=node["properties"]["v-name"], country=node["properties"]["v_country_of_citizenship"])

def name_country_label_mapping(index, node):
    return "{name} ({country})".format(name=node["properties"]["label"], country=node["properties"]["v_country_of_citizenship"].replace(':',''))

def edge_label_mapping(index, edge):
    label = edge["properties"]["e_relation"].replace('_', ' ')
    if label != "common consecration":
       return label
    else:
       return

w.set_node_label_mapping(name_country_label_mapping)
w.set_edge_label_mapping(edge_label_mapping)

def heatmap(element):
#    return element['properties'].get('v_pagerank', 0) * 50
#    return element['properties'].get('v_wahrscheinlichkeit', 0) * 100
#    return element['properties'].get('v_degree', 0) * 100
#    return element['properties'].get('v_betweenness', 0) * 100
    return element['properties'].get('v_papabile_index', 0) * 200

def edge_color_mapping(index, edge):
    type = edge["properties"]["e_relation"]
    if type == "common_consecration":
      return
    elif type == "co_consecrator":
      return 'darkmagenta'
    elif type == "consecrator":
      return 'darkslateblue'

def edge_direction_mapping(index, edge):
    type = edge["properties"]["e_relation"]
    if type == "common_consecration":
      return 0
    elif type == "co_consecrator":
      return 0
    elif type == "consecrator":
      return 1

def node_color_mapping(index, node):
    type = node["properties"]["v_country_of_citizenship"]
    if type == ":IT:":
      return 'green'
    elif node['id'] == "n909":
      return '#fbf334'
    return 'indianred'



w.set_heat_mapping(heatmap)

w.hierarchic_layout()
w.set_edge_color_mapping(edge_color_mapping )
w.set_directed_mapping(edge_direction_mapping)
w.set_node_color_mapping(node_color_mapping)

w.set_sidebar(enabled=True)
w.set_sidebar(start_with='Neighborhood')
w.set_neighborhood(3, ['n909'])
w

GraphWidget(layout=Layout(height='800px', width='100%'))